# Sentencepiece text classification
Using sp tokenized texts to experiment with cluster and classify text

In [1]:
import krp_ngram as k
import sentencepiece as spm
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import networkx as nx
import pandas as pd
import codecs, os, re
from collections import Counter
from collections import defaultdict
from operator import itemgetter
import numpy as np
import community

Import the texts from kr-shadow, prepare sp_lists

In [ ]:
sp = spm.SentencePieceProcessor()
sp.load("model/m0-krp-30000.model")
krshadow="/home/chris/projects/kr-shadow/"
# mapping from kr number to full title 
kr2tit={}
tsize = 1048576*5
bu=[a for a in os.listdir(krshadow) if a.startswith("KR")]
bu.sort()
for l in bu:
    files = [a for a in os.listdir("%s%s" % (krshadow, l)) if a.startswith("KR5")]
    files.sort()
    #KR1f0004 孝經注疏-唐-玄宗.txt
    
    lx = 0
    kt = codecs.open("kr2titles.txt", "w", "utf-8")
    for f in files:
            print(f)
            kr, tit = f[:-4].split(" ", 1)
            kr2tit[kr]=tit
            ct = ""
            for line in codecs.open("%s%s/%s" % (krshadow, l, f)).readlines():
                line = line.replace("\n", "")
                line = line.replace("　", "")
                if line.startswith("*"):
                    continue
                if line.startswith("#"):
                    continue
                line = re.sub(r"[\u3000-\u3fff\uff00-\uffff()/]", "", line)
                ct += line
            if len(ct) > tsize:
                c = Counter()
                for i in range(0, len(ct), tsize):
                    c.update(ct[i:i+tsize])
            else:
                c = Counter(sp.encode_as_pieces(ct))
            total = sum(c.values())
            cs=sum([(len(a[0]*a[1])) for a in c.items()])
            kt.write("%s %s: length:%d vocsize:%d csum:%d ctlen:%d\n" % (kr, tit, total, len(c), cs, len(ct)))
            of=codecs.open("text1/%s.voc" % (kr), "w", "utf-8")
            of.write("# %s\t%s\t%d\t%d\n" % (kr, tit, total, len(c)))
            for a in c.most_common():
                of.write("%s\t%8.8f\n" % (a[0], a[1] / total))
            of.close()
    kt.close()

In [2]:
def agg_voclist(tdir="text"):
    """    returns a dictionary of lists, one entry for each sp (with score option):
    {'好施' :
    [("KR1a0001", -11.637325286865234),
    ("KR2a0007", -11.83456039428711)]}
    """
    vx=defaultdict(list)
    mv=os.listdir(tdir)
    mv.sort()
    for f in mv:
        krpid=f[:-4]
        for line in codecs.open("%s/%s" % (tdir, f), "r", "utf-8"):
            if line.startswith("#"):
                continue
            v,p = line[:-1].split("\t",1)
            vx[v].append((krpid, p))
    return vx


In [3]:
vx = agg_voclist()

In [ ]:
if 1:
    labels, values = zip(*Counter([len(a[1]) for a in vx.items()]).items())
    cut=10
    width=10
    plt.title("SP Histogram")
    plt.ylabel("Count")
    plt.xlabel("No of texts")
    indexes = np.arange(cut, len(labels))
    plt.bar(indexes, values[cut:], width)
plt.savefig("text-hist2.png")
plt.show()


In [ ]:
vt=defaultdict(list)
vk=list(vx.keys())
for v1 in vk[0:10]:
    vt[v1]=vx[v1]


In [ ]:
vx[vk[9]]

In [ ]:
# for every text, we have a key of another text and the weight of the connection
tl = defaultdict(lambda: defaultdict(int))
skip=1
if 1:
    vk=list(vx.keys())
    for v1 in vk:
        cur = vx[v1]
        if len(cur) > skip:
            for i, c1 in enumerate(cur):
                for j, c2 in enumerate(cur):
                    if i != j:
                        w = float(c1[1]) + float(c2[1])
                        tl[c1[0]][c2[0]] += w
                        tl[c2[0]][c1[0]] += w
        

In [ ]:
tk1 = list(tl.keys())
tk1.sort()
of=codecs.open("tkl.txt", "w", "utf-8")
for tk in tk1:
    ts = sorted(tl[tk].items(), key=lambda x : x[1], reverse=True)
    
    of.write (tk, ",".join(["%s:%f" % a for a in ts[0:10]]), "\n")

In [14]:
def vocmatrix(vx, tdir="text", up=1000, lo=10):
    """Converts the aggregated voclist into a matrix."""
    bud=defaultdict(lambda: defaultdict(list))
    mv=[a for a in os.listdir(tdir) if a.endswith("voc")]
    for v in vx:
        vg=vx[v]
        #print (v,len(vg))
        if len(vg) > lo and len(vg) < up:
            t1 = vg[0]
            for t1 in vg:
                for t2 in vg:
                #print (t1, t2)
                    k1 = t1[0]
                    v1 = t1[1]
                    k2 = t2[0]
                    v2 = t2[1]
                    if k1 != k2:
                        if k1 < k2:
                            bud[k1][k2].append(float(v1) + float(v2))
                        else:
                            bud[k2][k1].append(float(v1) + float(v2))
    return bud


In [ ]:
sx =vocmatrix(vx, up=1000, lo=500)

In [ ]:
if 1:
    o = []
    G = nx.Graph()
    for t in sx:
        for s in sx[t]:
            #t, s, sx[t][s]
            o.append((t, s, sum(sx[t][s])))
    G.add_weighted_edges_from(o)
#    for g in G.nodes:
#        G.nodes[g]['label']=k.krp_names[g]
#    G.graph["name"] = name
    nx.write_gexf(G, "text-w.gexf")
    #partition = community.best_partition(G)
    #size = float(len(set(partition.values())))


In [10]:
for s in sx[t]:
    print (t, s, sx[t][s])

KR5a0010 KR5a0220 [0.00014497000000000002]
KR5a0010 KR5c0383 [0.00019470000000000002]
KR5a0010 KR5b0258 [0.0006367199999999999]
KR5a0010 KR5g0089 [0.0006422000000000001]
KR5a0010 KR5b0210 [0.00011788]
KR5a0010 KR5b0321 [0.00017338]
KR5a0010 KR5b0150 [0.00011538000000000001]
KR5a0010 KR5a0225 [0.00018072]
KR5a0010 KR5g0021 [0.00030751]
KR5a0010 KR5e0005 [0.00025731]
KR5a0010 KR5e0008 [0.00137672]


In [ ]:
for s in sx[t]:
    print (t, s, sx[t][s])

In [ ]:
import community
partition = community.best_partition(G)
size = float(len(set(partition.values())))
pos = nx.spring_layout(G)
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()


In [ ]:
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.savefig("text-nw-1.png")

In [ ]:
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    list_nodes.sort()
    pc = Counter([a[0:3] for a in list_nodes])
    pcs = sum(pc.values())
    print (com, pcs, "\t".join(["%s:%3.3d" % (a[0], a[1]) for a in pc.most_common()]))

In [ ]:
len(partition)

In [ ]:
sp = spm.SentencePieceProcessor()
sp.load("model/m0-krp-30000.model")

p="欽定四庫全書經部九皇祐新樂圖記樂類提要(臣)等謹案皇祐新樂圖記三卷宋阮逸胡瑗奉勅撰仁宗景祐三年二月以李照樂穿鑿特詔較定鍾律依周禮及厯代史志立議笵金至皇祐五年樂成奏上此其圖記也舊本從明文淵閣録出後有宋陳振孫嘉定己亥跋云借虎邱寺本録葢當時所賜藏之名山者又有元天厯二年呉夀民跋明萬厯三十九年趙開美跋叙是書源委頗詳考初置局時逸瑗與房庶等皆驛召豫議詔諸家各作鍾律以獻而持論互異司馬光主逸瑗之說范鎮主房庶之說往反争議卒不能以相一其往返書牘具光傳家集中而鎮所作東齋記事亦畧存其㮣大抵逸瑗以為黄鍾之管積八百一十分容一千二百黍又以九章圜田算法計之黄鍾管每長一分積九分容十三黍三分黍之一空徑三分四釐六毫圍十分三釐八毫圍徑用徑三圍九古率而改圍九分為九方分則遷就之術也司馬光曰古律已亡非黍無以見度非度無以見律律不生於度與黍將何從生非謂太古以來律必生於度也特以近世古律不存故反從度法求之耳其論最明范鎮譏其以度起律誠為過當然鎮以秬黍律尺龠鬴斛算數權衡鐘磬十者必相合而不相戾然後為得亦不為無見也以律起度與以度起律源流本無異同而二家算術不精逸瑗等得之於横黍而失之於圍徑又以大黍累尺小黍實管自相乖反房庶以千二百黍實之管中隨其長短截之以為九寸之管取三分以度空徑則空徑不生於黍而别有一物為度以起分竟不必實黍於管未見其為通論也是書上卷具載律吕黍尺四量權衡之法皆以横黍起度故樂聲失之於髙中下二卷攷定鐘磬晉鼓及三牲鼎鸞刀制度則精核可取云乾隆四十六年十月恭校上總纂官(臣)紀昀(臣)陸錫熊(臣)孫士毅總校官(臣)陸費墀欽定四庫全書皇祐新樂圖記卷上宋阮逸胡瑗撰律度量衡四圖總叙詔㫖篇第一皇上睿敏聰明自天所禀精志典學而大曉雅音景祐中親製樂章乾安等十曲播于樂府用于郊廟皇祐二年秋九月大饗明堂復製樂章成安等十有四曲先詔太常調習鐘律奏御日上謂侍臣曰鎛鐘特磬未協音律宜改制"
c = Counter(sp.encode_as_pieces(p))
cs=sum([(len(a[0]*a[1])) for a in c.items()])
print(cs, len(p))

In [7]:
sxk = list(sx.keys())
len(sxk)

1125

In [11]:
labels, values = zip(*Counter([len(a[1]) for a in sx.items()]).items())    

In [13]:
labels

(6683,
 3075,
 1032,
 10,
 11,
 12,
 7181,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 1047,
 24,
 1049,
 26,
 1051,
 28,
 29,
 30,
 31,
 32,
 33,
 1058,
 35,
 3108,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 1071,
 48,
 49,
 50,
 51,
 2100,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 36,
 1088,
 66,
 67,
 69,
 70,
 71,
 72,
 4169,
 74,
 75,
 1100,
 77,
 13,
 2128,
 5201,
 82,
 83,
 84,
 85,
 87,
 88,
 1113,
 1114,
 91,
 92,
 1117,
 95,
 96,
 97,
 98,
 99,
 100,
 870,
 102,
 103,
 3038,
 106,
 359,
 108,
 109,
 110,
 1135,
 113,
 114,
 115,
 116,
 3262,
 2167,
 120,
 1145,
 122,
 2171,
 3197,
 126,
 2069,
 128,
 3201,
 130,
 131,
 2181,
 2182,
 1159,
 2184,
 519,
 1162,
 1163,
 140,
 194,
 1166,
 1198,
 144,
 5265,
 2194,
 147,
 149,
 150,
 151,
 153,
 154,
 2203,
 1744,
 4253,
 2767,
 159,
 4131,
 162,
 27,
 1188,
 5148,
 6314,
 1195,
 173,
 174,
 1199,
 176,
 76,
 178,
 2229,
 1206,
 1209,
 187,
 5308,
 2238,
 191,
 192,
 5152,
 2242,
 1219,
 196,
 1221,
 200,
 202,
